In [2]:
import sqlite3


conn = sqlite3.connect(':memory:')  


cursor = conn.cursor()


cursor.execute('''
CREATE TABLE Weather (id INTEGER PRIMARY KEY,recordDate DATE,temperature INTEGER)''')


data = [(1, '2015-01-01', 10),(2, '2015-01-02', 25),(3, '2015-01-03', 20),(4, '2015-01-04', 30)
]
cursor.executemany('INSERT INTO Weather (id, recordDate, temperature) VALUES (?, ?, ?)', data)
conn.commit()


query = '''
SELECT w1.id
FROM Weather w1
JOIN Weather w2 ON DATE(w1.recordDate, '-1 day') = w2.recordDate
WHERE w1.temperature > w2.temperature
'''

cursor.execute(query)
result = cursor.fetchall()


print("+----+\n| id |\n+----+")
for row in result:
    print(f"|  {row[0]} |")
print("+----+")


for row in result:
    cursor.execute('SELECT recordDate, temperature FROM Weather WHERE id = ?', (row[0],))
    current = cursor.fetchone()
    cursor.execute('SELECT temperature FROM Weather WHERE recordDate = DATE(?, "-1 day")', (current[0],))
    previous = cursor.fetchone()
    print(f"In {current[0]}, the temperature was higher than the previous day ({previous[0]} -> {current[1]}).")


conn.close()

+----+
| id |
+----+
|  2 |
|  4 |
+----+
In 2015-01-02, the temperature was higher than the previous day (10 -> 25).
In 2015-01-04, the temperature was higher than the previous day (20 -> 30).
